Imports of pandas, seaborn

In [ ]:
import pandas as pd
import seaborn as sns
from itertools import islice
import nltk
from nameparser.parser import HumanName
import time
import csv

Download necessary ressources

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

Read train dataset from csv file

In [ ]:
df = pd.read_csv("../data/train.csv")

Statistical analysis about length of every "text" in dataset - For loop + Sorting by longest articles

Generate graph of average length of articles in 100 of characters

In [ ]:
content_length = {}
for content in df["text"]:
    try:
        if len(content) in content_length:
            content_length[len(content)]+=1
        else:
            content_length[len(content)] = 1
    except: # Error in the dataset, one of the entries seems to be a float. To prevent execution stop: try/except.
        pass

content_length = dict(reversed(sorted(content_length.items(), key=lambda item: item[0]))) 

values = {}

for loop in content_length.items():
    if loop[0]//100 in values:
        values[loop[0]//100]+=1
    else:
        values[loop[0]//100] = 1

my_df = pd.DataFrame(values.items())
ax = sns.barplot(x=0, y=1, data=my_df)
ax.set(xlabel = '100 of characters', ylabel='Number of articles', title='Length of articles in 100 of characters')


Statistical analysis about length of every fake news (label = 1) in dataset - For loop + Sorting by longest articles

Generate graph of average length of articles in 100 of characters

In [ ]:
content_length = {}
for content in df.loc[df['label'] == 1]["text"]:
    try:
        if len(content) in content_length:
            content_length[len(content)]+=1
        else:
            content_length[len(content)] = 1
    except: # Error in the dataset, one of the entries seems to be a float. To prevent execution stop: try/except.
        pass

content_length = dict(reversed(sorted(content_length.items(), key=lambda item: item[0]))) 

values = {}

for loop in content_length.items():
    if loop[0]//100 in values:
        values[loop[0]//100]+=1
    else:
        values[loop[0]//100] = 1

my_df = pd.DataFrame(values.items())
ax = sns.barplot(x=0, y=1, data=my_df)
ax.set(xlabel = '100 of characters', ylabel='Number of articles', title='Length of fake news in 100 of characters')


Statistical analysis about length of every non fake news (label = 0) in dataset - For loop + Sorting by longest articles

Generate graph of average length of articles in 100 of characters

In [ ]:
content_length = {}
for content in df.loc[df['label'] == 0]["text"]:
    try:
        if len(content) in content_length:
            content_length[len(content)]+=1
        else:
            content_length[len(content)] = 1
    except: # Error in the dataset, one of the entries seems to be a float. To prevent execution stop: try/except.
        pass

content_length = dict(reversed(sorted(content_length.items(), key=lambda item: item[0]))) 

values = {}

for loop in content_length.items():
    if loop[0]//100 in values:
        values[loop[0]//100]+=1
    else:
        values[loop[0]//100] = 1

my_df = pd.DataFrame(values.items())
ax = sns.barplot(x=0, y=1, data=my_df)
ax.set(xlabel = '100 of characters', ylabel='Number of articles', title='Length of fake news in 100 of characters')

Check amount of names and save result to CSV file.

In [ ]:
def get_human_names(text):
    tokens = nltk.tokenize.word_tokenize(text)
    pos = nltk.pos_tag(tokens)
    sentt = nltk.ne_chunk(pos, binary = False)
    person_list = []
    person = []
    name = ""
    for subtree in sentt.subtrees(filter=lambda t: t.label() == 'PERSON'):
        for leaf in subtree.leaves():
            person.append(leaf[0])
        if len(person) > 1: #avoid grabbing lone surnames
            for part in person:
                name += part + ' '
            if name[:-1] not in person_list:
                person_list.append(name[:-1])
            name = ''
        person = []

    return (person_list)

start_time = time.time()

f = open("../data/name_output.csv", 'w', encoding='UTF8', newline='')
writer = csv.writer(f)
writer.writerow(["article_id","nb_names"])

for id, content in enumerate(df["text"]):
    if id%100==0:
        print(id)
    try:
        writer.writerow([id, len(get_human_names(content))])
    except:
        pass

print(time.time()-start_time)
f.close()
